In [1]:
import pandas as pd
import numpy as np
import datetime
import glob
import re 
import os

In [2]:
#makes floats diplay commas and two decimals
pd.options.display.float_format = '{:,.2f}'.format

# makes ints display commas 
class _IntArrayFormatter(pd.io.formats.format.GenericArrayFormatter):

    def _format_strings(self):
        formatter = self.formatter or (lambda x: ' {:,}'.format(x))
        fmt_values = [formatter(x) for x in self.values]
        return fmt_values

pd.io.formats.format.IntArrayFormatter = _IntArrayFormatter

pd.set_option('display.max_columns', None)

## Scraping the Links
I'm scraping all of the Voter Registration Statistics table rows and putting the date, title, and link into the vr_tbl.csv file. I'm going to use the file to loop through the links to get to the files.

In [3]:
from requests_html import HTML, HTMLSession
import csv
import urllib

In [4]:
# csv_file = open('vr_tbl.csv', 'w', newline='')
# csv_writer = csv.writer(csv_file)
# csv_writer.writerow(['date', 'title', 'link'])

# Go to https://elections.wi.gov/index.php/publications/statistics/registration
session = HTMLSession()
r = session.get('https://elections.wi.gov/index.php/publications/statistics/registration')

In [5]:
table = r.html.find('tbody', first=True)

In [7]:
rows = table.find('tr')

In [11]:
print(rows[0].text)

March 1, 2021 Voter Registration Statistics
03/01/2021


In [13]:
print(rows[0].find('time', first=True).text)

03/01/2021


In [14]:
print(rows[0].find('a', first=True).text)

March 1, 2021 Voter Registration Statistics


In [17]:
most_recent = rows[0].find('a', first=True).attrs['href']

In [9]:
for row in rows:
    date = row.find('time', first=True).text
    print(date)
    
    title = row.find('a', first=True).text
    print(title)
    
    link = row.find('a', first=True).attrs['href']
    print(link)
    
    

03/01/2021
March 1, 2021 Voter Registration Statistics
/node/7360
02/01/2021
February 1, 2021 Voter Registration Statistics
/node/7331
01/01/2021
January 1, 2021 Voter Registration Statistics
/node/7301
12/01/2020
December 1, 2020 Voter Registration Statistics
/node/7265
11/01/2020
November 1, 2020 Voter Registration Statistics
/node/7220
10/01/2020
October 1, 2020 Voter Registration Statistics
/node/7147
09/01/2020
September 1, 2020 Voter Registration Statistics
/node/7070
08/01/2020
August 1, 2020 Voter Registration Statistics
/node/6993
07/01/2020
July 1, 2020 Voter Registration Statistics
/node/6948
06/01/2020
June 1, 2020 Voter Registration Statistics
/node/6919
05/01/2020
May 1, 2020 Voter Registration Statistics
/node/6886
04/01/2020
April 1, 2020 Voter Registration Statistics
/node/6830
03/01/2020
March 1, 2020 Voter Registration Statistics
/node/6701
02/01/2020
February 1, 2020 Voter Registration Statistics
/node/6677
01/01/2020
January 1, 2020 Voter Registration Statistics
/n

In [10]:
csv_writer.writerow([date, title, link])
    
csv_file.close()

## Loop to Download the Files

In [11]:
starter_url = 'http://elections.wi.gov'
date_pattern = re.compile(r'(\d{2})/(\d{2})/(\d{4})')

In [12]:
import time
from csv import reader

In [13]:
with open('vr_tbl.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    header = next(csv_reader)
    for row in csv_reader:
        date = date_pattern.sub(r'\3\1\2', row[0])
        path = row[2]
        url = starter_url + path

        month = session.get(url)

        table = month.html.find('tbody', first=True)

        file_rows = table.find('tr')
        for file_row in file_rows:
            title = file_row.find('a', first=True).text
            #print(title)
            file = file_row.find('a', first=True).attrs['href']\
            
            if re.search(r'xlsx', title):
                if re.search(r'[c|C]ounty', title):
                    #print(title)
                    f = session.get(file)
                    with open('county\\{}.xlsx'.format(date), 'wb') as outfile:
                        outfile.write(f.content)
                elif re.search(r'[w|W]ard', title):
                    #print(title)
                    f = session.get(file)
                    with open('muni\\{}.xlsx'.format(date), 'wb') as outfile:
                        outfile.write(f.content)
                elif re.search(r'[c|C]ong', title):
                    f = session.get(file)
                    with open('congressional\\{}.xlsx'.format(date), 'wb') as outfile:
                        outfile.write(f.content)
                elif re.search(r'[a|A]ssembly', title):
                    f = session.get(file)
                    with open('assembly\\{}.xlsx'.format(date), 'wb') as outfile:
                        outfile.write(f.content)
                elif re.search(r'[s|S]enate', title):
                    f = session.get(file)
                    with open('senate\\{}.xlsx'.format(date), 'wb') as outfile:
                        outfile.write(f.content)
                elif re.search(r'[a|A]ge', title):
                    f = session.get(file)
                    with open('age group\\{}.xlsx'.format(date), 'wb') as outfile:
                        outfile.write(f.content)
                elif re.search(r'[s|S]ummary', title):
                    f = session.get(file)
                    with open('summary\\{}_Voter_Registration_Summary_Statistics.xlsx'.format(date), 'wb') as outfile:
                        outfile.write(f.content)
            time.sleep(1)
        #time.sleep(2)

## Starting the Timeline
Using 20160430_Voter_Registration_Summary_Statistics.xlsx to start the VR timeline. BTW, Voters by Senate tab has a trailing space in it. Just delete the space in the file within Excel

In [127]:
# meant for County, Congressional, Senate, and Assembly
def start(geography):
    # Voters by Senate tab has a trailing space in it. Just delete the space in the file within Excel
    start = pd.read_excel('summary\\20160430_Voter_Registration_Summary_Statistics.xlsx', 'Voters by {}'.format(geography))

    start.rename(columns={'05/01/20152': '05/01/2015'}, inplace=True)
    start.rename(columns={'42,461':'01/04/2016'}, inplace=True)
    start.set_index(start.columns[0], inplace=True)
    
    if start.index.name == 'Date':
        start = start.transpose()
    
    start.dropna(axis='columns', how='all', inplace=True)
    start.columns = pd.to_datetime(start.columns)
    start = start[start.columns.sort_values()]
    
    # extract district numbers
    # exclude Age Group bc it has numbers for its groups
    if (geography != 'Age Group') and (re.search(r'\d+', start.index[3])): # use 'and' instead of '&'
        start.index = start.index.str.extract(r'(\d+)')[0]
        start.index = start.index.astype(str).str.zfill(2)
    return start

## Looping through Files

In [117]:
# use county, state_senate, state_assembly
def end(folder):
    end_files = glob.glob('{}\\*'.format(folder))

    dfs = []
    for file in end_files:
        end = pd.read_excel(file)
        # for each temp df, drop N/A columns
        if folder == 'age group':
            end.dropna(axis='index', thresh=2, inplace=True)
        end.dropna(axis='columns', how='all', inplace=True)

        # find header
        header_index = end[end.columns[0]].first_valid_index()
        
        # age group rly gave me a hard time
        if folder == 'age group':
            header = end.iloc[0]
            end.columns = header
        else:
            if header_index != 0: #checks for useless top rows
                end = end[header_index:] #splice off useless top rows

                #drop N/A rows and columns
                end.dropna(axis='index', how='all', inplace=True) 
                end.dropna(axis='columns', how='all', inplace=True) 

                #set header
                if end.iloc[0].isnull().values.any(): #senate data gave me a hard time
                    end.drop(header_index, axis=0, inplace=True)
                header = end.iloc[0]
                end.columns = header 
                end = end[1:]

        # drop code col and append to dfs
        #print(end)
        if folder == 'county':
            end = end.drop(end.columns[0], axis=1)
            end.set_index(end.columns[0], inplace=True)
        elif folder == 'age group':
            end.set_index(end.columns[0], inplace=True)
        else:
            end.set_index(end.columns[0], inplace=True)
            # extract district numbers
            end.index = end.index.str.extract(r'(\d+)')[0]
            end.index = end.index.astype(str).str.zfill(2)
        #print(end)
        end = end.iloc[:,-1:]
        end.rename(columns={end.columns[0]:re.findall(r'\d+', file)[0]}, inplace=True)
        
        dfs.append(end)

    #print(dfs)
    all_end = pd.concat(dfs, axis=1, join='outer')
    all_end.dropna(axis='index', how='any', inplace=True)

    all_end.columns = pd.to_datetime(all_end.columns)
    all_end = all_end[all_end.columns.sort_values()]
    
#     # extract district numbers
#     if (folder != 'age group') and re.search(r'\d+', all_end.index[3]):
#         all_end.index = all_end.index.str.extract(r'(\d+)')[0]
    
    st = start(folder.title())
    
    # makes joining senate data possible
    st.index.name = folder.title()
    all_end.index.name = folder.title()
   
    end = pd.merge(st, all_end, on=all_end.index.name)

    end = end[end.columns.sort_values()]
    
    # remove timestamps
    end.columns = pd.to_datetime(end.columns).date
    
    # remove monthly dupes
    end = end.loc[:, ~pd.to_datetime(end.columns).strftime('%y-%m').duplicated()]
    
    end.loc['*Total',:] = end.sum(axis=0).astype(int)
    
    end.sort_index(inplace=True)
        
    # make everything into a float so that diff() works
    end = end.astype(float)

    return end

In [128]:
cty = end('county')
sd = end('senate')
ad = end('assembly')
cd = end('congressional')
age = end('age group')

In [130]:
cty.to_csv('upload2gsheet\\vr_county.csv')
sd.to_csv('upload2gsheet\\vr_senate.csv')
ad.to_csv('upload2gsheet\\vr_assembly.csv')
cd.to_csv('upload2gsheet\\vr_congressional.csv')
age.to_csv('upload2gsheet\\vr_age.csv')

In [131]:
d2g.upload(cty, spreadsheet_key, 'County', credentials=credentials, row_names=True)
d2g.upload(cd, spreadsheet_key, 'CD', credentials=credentials, row_names=True)
d2g.upload(sd, spreadsheet_key, 'SD', credentials=credentials, row_names=True)
d2g.upload(ad, spreadsheet_key, 'AD', credentials=credentials, row_names=True)
d2g.upload(age, spreadsheet_key, 'Age', credentials=credentials, row_names=True)

<Worksheet 'Age' id:211692979>

### Municipalities

In [132]:
# keys are meant for attaching the geography's name to the merged df
muni_key = pd.read_csv('muni_key.csv')
muni_key['Hindi'] = muni_key['Hindi'].astype(int)
muni_key['Hindi'] = muni_key['Hindi'].astype(str)
muni_key['Hindi'] = muni_key['Hindi'].str.zfill(5)
muni_key.set_index('Hindi', inplace=True)

In [133]:
files = glob.glob('muni\\*')

In [134]:
dfs = []
for fn in files:
    file = pd.read_excel(fn)
    # for each temp df, drop N/A columns
    file.dropna(axis='columns', how='all', inplace=True)
    
    # find header
    header_index = file[file.columns[0]].first_valid_index()
    
    if header_index != 0: #checks for useless top rows
        file = file[header_index:] #splice off useless top rows

        #drop N/A rows and columns
        file.dropna(axis='index', how='all', inplace=True) 
        file.dropna(axis='columns', how='all', inplace=True) 

        #set header
        #file.drop(header_index, axis=0, inplace=True)
        header = file.iloc[0]
        file.columns = header 
        file.dropna(axis='index', how='all', inplace=True) 
      
        if (file.columns.to_series() == file.iloc[0]).all():
            file = file[1:]
    else:
        if (file.columns.str.contains('^Unnamed')).any():
            file.columns = file.iloc[0]
            file = file[1:]
    
    
    file.set_index(file.filter(regex=(r'(Hindi)|(HINDI)')).columns[0], inplace=True)
    file.index.name = file.index.name.title()
    file.index = file.index.astype(int)
    file.index = file.index.astype(str)
    file.index = file.index.str.zfill(5)
    
    file.rename(columns={file.columns[-1]:re.findall(r'\d+', fn)[0]}, inplace=True)
    
    #file = file[file.columns[-1]]
    file = file.groupby(['Hindi']).aggregate({file.columns[-1]:'sum'})

    dfs.append(file)

In [135]:
all_mn = pd.concat(dfs, axis=1, join='outer')

all_mn.columns = pd.to_datetime(all_mn.columns).date
all_mn = all_mn[all_mn.columns.sort_values()]

# remove monthly dupes
all_mn = all_mn.loc[:, ~pd.to_datetime(all_mn.columns).strftime('%y-%m').duplicated()]

# make everything into a float so that diff() works
all_mn = all_mn.astype(float)

In [136]:
muni = pd.concat([muni_key, all_mn], axis=1)
muni = muni.sort_index()

In [137]:
muni.loc['*Total',:] = muni.sum(axis=0).astype(int)
muni['*Mean'] = muni.mean(axis=1).astype(int)
muni.sort_index(inplace=True)

In [138]:
muni.sort_values(by='*Mean', ascending=False, inplace=True)

In [139]:
muni['row_num'] = range(len(muni))

muni['Muni'] = muni['row_num'].astype(str).str.zfill(4) + '. ' + muni['Muni']

muni.drop(columns = ['*Mean', 'row_num'], inplace=True)

In [140]:
d2g.upload(muni, spreadsheet_key, 'Municipality', credentials=credentials, row_names=True)

<Worksheet 'Municipality' id:575947317>

In [141]:
muni.to_csv('upload2gsheet\\vr_muni.csv')

## Calculating Changes

In [142]:
cty_diff = cty.diff(axis=1)
sd_diff = sd.diff(axis=1)
ad_diff = ad.diff(axis=1)
cd_diff = cd.diff(axis=1)
age_diff = age.diff(axis=1)

In [143]:
cty_diff.to_csv('upload2gsheet\\vrdiff_county.csv')
sd_diff.to_csv('upload2gsheet\\vrdiff_senate.csv')
ad_diff.to_csv('upload2gsheet\\vrdiff_assembly.csv')
cd_diff.to_csv('upload2gsheet\\vrdiff_congressional.csv')
age_diff.to_csv('upload2gsheet\\vrdiff_age.csv')

In [144]:
muni_diff = muni.iloc[:, 2:].diff(axis=1)

In [145]:
muni_diff = pd.concat([muni_key, muni_diff], axis=1)
muni_diff = muni_diff.sort_index()
muni_diff = pd.concat([muni_diff.iloc[[len(muni_diff)-1],:], muni_diff.drop('*Total', axis=0)], axis=0, copy=False)

In [146]:
muni_diff.to_csv('vrdiff_muni.csv')

In [147]:
muni_diff

,County,Muni,2013-06-30,2013-07-31,2013-09-17,2013-11-04,2014-08-31,2014-09-30,2014-10-31,2014-11-30,2014-12-31,2015-01-31,2015-02-28,2015-03-31,2015-04-30,2015-05-31,2015-06-30,2015-07-31,2015-09-30,2015-10-31,2016-01-03,2016-03-24,2016-05-31,2016-06-30,2016-07-31,2016-08-31,2016-09-30,2016-10-18,2016-11-01,2016-12-06,2017-01-31,2017-02-28,2017-04-02,2017-05-31,2017-06-30,2017-07-31,2017-08-31,2017-09-30,2017-10-31,2017-11-30,2017-12-31,2018-01-31,2018-02-28,2018-04-01,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,2018-11-30,2019-01-01,2019-02-01,2019-04-01,2019-05-01,2019-06-01,2019-07-01,2019-09-03,2019-10-01,2019-11-01,2019-12-01,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01,2021-01-01,2021-02-01
Hindi,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
72291,WOOD COUNTY,CITY OF WISCONSIN RAPIDS - WOOD COUNTY,NaN,-16.00,-20.00,-29.00,-40.00,-7.00,16.00,209.00,-40.00,-20.00,-24.00,-21.00,-7.00,-7.00,-12.00,-313.00,-11.00,-14.00,-21.00,53.00,365.00,-20.00,5.00,-19.00,27.00,63.00,79.00,849.00,-328.00,-28.00,-34.00,-20.00,-416.00,-551.00,0.00,-8.00,-7.00,-190.00,0.00,-649.00,31.00,57.00,29.00,14.00,67.00,109.00,58.00,192.00,461.00,-28.00,-69.00,-33.00,50.00,-32.00,0.00,-270.00,37.00,-31.00,-11.00,-4.00,1.00,63.00,274.00,82.00,-9.00,-33.00,28.00,188.00,337.00,295.00,409.00,-11.00,-37.00
01002,ADAMS COUNTY,TOWN OF ADAMS - ADAMS COUNTY,NaN,0.00,0.00,-1.00,-6.00,-1.00,-27.00,-15.00,33.00,0.00,-2.00,-8.00,1.00,0.00,0.00,-18.00,-1.00,0.00,0.00,6.00,33.00,1.00,-12.00,-12.00,0.00,5.00,7.00,54.00,2.00,-8.00,-1.00,0.00,0.00,-86.00,0.00,0.00,0.00,4.00,0.00,-53.00,-3.00,3.00,3.00,10.00,3.00,-2.00,12.00,5.00,62.00,-4.00,-3.00,-2.00,1.00,-1.00,0.00,-4.00,0.00,0.00,-10.00,1.00,-1.00,4.00,8.00,1.00,0.00,2.00,0.00,13.00,25.00,33.00,57.00,-4.00,-5.00
01004,ADAMS COUNTY,TOWN OF BIG FLATS - ADAMS COUNTY,NaN,0.00,-8.00,-3.00,-3.00,-1.00,0.00,19.00,0.00,-4.00,-1.00,-1.00,6.00,0.00,0.00,-27.00,0.00,0.00,0.00,-2.00,16.00,0.00,-9.00,-1.00,-1.00,-2.00,-1.00,56.00,-2.00,-2.00,-2.00,-4.00,-22.00,-43.00,-2.00,0.00,0.00,5.00,-9.00,-19.00,-2.00,0.00,11.00,2.00,-1.00,1.00,0.00,-1.00,35.00,1.00,0.00,-3.00,7.00,-5.00,0.00,-15.00,-2.00,0.00,0.00,1.00,-3.00,5.00,7.00,-1.00,-5.00,-3.00,-3.00,22.00,14.00,6.00,70.00,-3.00,-3.00
01006,ADAMS COUNTY,TOWN OF COLBURN - ADAMS COUNTY,NaN,0.00,-1.00,0.00,1.00,0.00,0.00,6.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,-9.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-1.00,-1.00,0.00,15.00,-2.00,0.00,-1.00,2.00,-1.00,-12.00,0.00,0.00,0.00,1.00,0.00,-6.00,2.00,0.00,3.00,0.00,0.00,-2.00,0.00,1.00,6.00,-3.00,0.00,-2.00,-2.00,-2.00,0.00,-7.00,0.00,2.00,-1.00,0.00,0.00,3.00,4.00,-4.00,1.00,0.00,-1.00,3.00,2.00,-1.00,16.00,-1.00,2.00
01008,ADAMS COUNTY,TOWN OF DELL PRAIRIE - ADAMS COUNTY,NaN,0.00,-3.00,-1.00,25.00,31.00,-18.00,17.00,-6.00,-4.00,1.00,0.00,3.00,0.00,0.00,-21.00,-1.00,-4.00,0.00,0.00,20.00,-3.00,1.00,-1.00,0.00,-2.00,4.00,77.00,-15.00,0.00,-5.00,-1.00,-10.00,-75.00,0.00,-2.00,0.00,0.00,4.00,-90.00,-2.00,1.00,5.00,2.00,10.00,6.00,8.00,13.00,78.00,-3.00,3.00,-11.00,2.00,-5.00,-2.00,-43.00,-1.00,-3.00,-1.00,0.00,0.00,6.00,12.00,10.00,-1.00,4.00,7.00,10.00,32.00,41.00,76.00,-3.00,-3.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72186,WOOD COUNTY,VILLAGE OF VESPER - WOOD COUNTY,NaN,0.00,-1.00,-1.00,-4.00,0.00,-2.00,12.00,0.00,-4.00,0.00,0.00,0.00,0.00,-1.00,-6.00,0.00,0.00,0.00,4.00,12.00,1.00,-1.00,-1.00,-1.00,-4.00,8.00,-7.00,27.00,-2.00,0.00,-2.00,0.00,-28.00,0.00,4.00,0.00,-8.00,1.00,-17.00,-1.00,0.00,-1.00,3.00,1.00,3.00,2.00,4.00,0.00,0.00,0.00,0.00,-1.00,0.00,0.00,-8.00,1.00,1.00,0.00,-1.00,-3.00,1.00,0.00,3.00,0.00,1.00,2.00,5.00,6.00,6.00,12.

In [148]:
d2g.upload(cty_diff, spreadsheet_key, 'County Shift', credentials=credentials, row_names=True)
d2g.upload(cd_diff, spreadsheet_key, 'CD Shift', credentials=credentials, row_names=True)
d2g.upload(sd_diff, spreadsheet_key, 'SD Shift', credentials=credentials, row_names=True)
d2g.upload(ad_diff, spreadsheet_key, 'AD Shift', credentials=credentials, row_names=True)
d2g.upload(age_diff, spreadsheet_key, 'Age Shift', credentials=credentials, row_names=True)

<Worksheet 'Age Shift' id:2112800033>

In [149]:
# have to reset_index due to an error
d2g.upload(muni_diff.reset_index(), spreadsheet_key, 'Municipality Shift', credentials=credentials, row_names=True)

<Worksheet 'Municipality Shift' id:142913538>

## Verticalize
Prepare dfs for Tableau visualization

In [150]:
def vert(hori):
    ver = hori.transpose().copy()
    
    if hori.columns[1] == 'Muni':
        ver.columns = ver.loc['Muni']
        ver = ver.iloc[2:]

        ver.rename(columns={np.nan: '*Total'}, inplace=True)
    
    ver.index.rename('Date', inplace=True)
    ver.columns = ver.columns.str.zfill(2)
    
    ver_diff = ver.diff()
    ver_pct = ver.pct_change()
    
    ver['Type'] = 'Population'
    ver_diff['Type'] = 'Shifts'
    ver_pct['Type'] = 'Shifts %'
    
    all_ver = pd.concat([ver, ver_diff, ver_pct])

    return all_ver

In [151]:
def stack(hori):
    ver = vert(hori)
    
    stack = ver.reset_index().melt(['Date', 'Type'])

    stack.set_index('Date', inplace=True)
    return stack

In [152]:
cty_ver = vert(cty)
sd_ver = vert(sd)
ad_ver = vert(ad)
cd_ver = vert(cd)
age_ver = vert(age)

In [153]:
muni_ver = vert(muni)

In [154]:
cty_stack = stack(cty)
sd_stack = stack(sd)
ad_stack = stack(ad)
cd_stack = stack(cd)
age_stack = stack(age)

In [155]:
priority_cty = ['MILWAUKEE COUNTY', 'DANE COUNTY', 'WAUKESHA COUNTY', 'OZAUKEE COUNTY', 'WASHINGTON COUNTY', 'BROWN COUNTY', 'OUTAGAMIE COUNTY', 'WINNEBAGO COUNTY', 'KENOSHA COUNTY', 'RACINE COUNTY']

In [156]:
cty_stack.loc[cty_stack['County'].isin(priority_cty), 'Key'] = 'Yes'

In [157]:
cty_stack.query('Key == "Yes"')

,Type,County,value,Key
Date,,,,
2009-07-01,Population,BROWN COUNTY,"139,194.00",Yes
2010-08-02,Population,BROWN COUNTY,"138,186.00",Yes
2010-12-28,Population,BROWN COUNTY,"141,849.00",Yes
2011-09-01,Population,BROWN COUNTY,"137,322.00",Yes
2011-11-01,Population,BROWN COUNTY,"137,099.00",Yes
...,...,...,...,...
2020-10-01,Shifts %,WINNEBAGO COUNTY,0.03,Yes
2020-11-01,Shifts %,WINNEBAGO COUNTY,0.02,Yes
2020-12-01,Shifts %,WINNEBAGO COUNTY,0.04,Yes


In [158]:
cty_stack.to_csv('tableau_prep\\vr_county_stack.csv')
sd_stack.to_csv('tableau_prep\\vr_senate_stack.csv')
ad_stack.to_csv('tableau_prep\\vr_assembly_stack.csv')
cd_stack.to_csv('tableau_prep\\vr_cd_stack.csv')
age_stack.to_csv('tableau_prep\\vr_age_stack.csv')

In [159]:
muni_stack = stack(muni)

In [160]:
priority_muni = ['0010. CITY OF JANESVILLE - ROCK COUNTY', '0009. CITY OF OSHKOSH - WINNEBAGO COUNTY', '0008. CITY OF RACINE - RACINE COUNTY', '0007. CITY OF APPLETON - MULTIPLE COUNTIES', '0006. CITY OF WAUKESHA - WAUKESHA COUNTY', '0005. CITY OF EAU CLAIRE - MULTIPLE COUNTIES', '0004. CITY OF KENOSHA - KENOSHA COUNTY', '0003. CITY OF GREEN BAY - BROWN COUNTY', '0002. CITY OF MADISON - DANE COUNTY', '0001. CITY OF MILWAUKEE - MULTIPLE COUNTIES']

In [161]:
muni_stack

,Type,Muni,value
Date,,,
2013-06-30,Population,*Total,"3,400,807.0"
2013-07-31,Population,*Total,"3,398,833.0"
2013-09-17,Population,*Total,"3,394,795.0"
2013-11-04,Population,*Total,"3,392,508.0"
2014-08-31,Population,*Total,"3,380,325.0"
...,...,...,...
2020-10-01,Shifts %,*Total,0.0
2020-11-01,Shifts %,*Total,0.0
2020-12-01,Shifts %,*Total,0.0


In [162]:
muni_stack.loc[muni_stack['Muni'].isin(priority_muni), 'Key'] = 'Yes'

In [163]:
muni_stack.query('Key == "Yes"')

,Type,Muni,value,Key
Date,,,,
2013-06-30,Population,0001. CITY OF MILWAUKEE - MULTIPLE COUNTIES,"321,577.0",Yes
2013-07-31,Population,0001. CITY OF MILWAUKEE - MULTIPLE COUNTIES,"321,459.0",Yes
2013-09-17,Population,0001. CITY OF MILWAUKEE - MULTIPLE COUNTIES,"320,907.0",Yes
2013-11-04,Population,0001. CITY OF MILWAUKEE - MULTIPLE COUNTIES,"320,681.0",Yes
2014-08-31,Population,0001. CITY OF MILWAUKEE - MULTIPLE COUNTIES,"318,186.0",Yes
...,...,...,...,...
2020-10-01,Shifts %,0010. CITY OF JANESVILLE - ROCK COUNTY,0.04,Yes
2020-11-01,Shifts %,0010. CITY OF JANESVILLE - ROCK COUNTY,0.03,Yes
2020-12-01,Shifts %,0010. CITY OF JANESVILLE - ROCK COUNTY,0.03,Yes


In [164]:
muni_stack.to_csv('tableau_prep\\vr_muni_stack.csv')